In [1]:
import pandas as pd
import numpy as np
from time import process_time
from joblib import dump
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from data_preprocessing import *
from model import *

In [2]:
training_path = "../dataset/original-dataset/marathi-training-data.csv"
training_data = read_data(training_path)
training_data

,text,label
0,"प्रा . प्रताप हरिदास : होय , मला वाटते की हा ए...",com_tech
1,"तर , विशिष्ट गोष्टींद्वारे , ठराविक कायद्यांद्...",bioche
2,- - - - - - - - - - - - - - - - - - - - - - - ...,cse
3,"तर , आपला अर्धा चिन्ह 9 वाजता असेल .",phy
4,"म्हणून , मी असे म्हणालो की जर शेकडो , हजारो कि...",phy
...,...,...
41992,"जरी आपण डेटा कूटबद्ध केला , तरीही हा मुख्य व्य...",cse
41993,"ते म्हणतात - "" ज्याला पाहण्यासाठी डोळे , ऎकण्य...",com_tech
41994,"प्रथम क्रोनोलॉजिकल , क्रॉनोलॉजी म्हणजे आपल्याल...",com_tech
41995,"त्या थोड्या तपशीलावर येईल , जेणेकरून संपूर्ण ग...",bioche


In [3]:
val_path = "../dataset/original-dataset/marathi-validation-data.csv"
val_data = read_data(val_path)
val_data

,text,label
0,1 ची ओळ .,cse
1,"तर , ही एक टॉवर आहे जी टॉवरवर निश्चित केली जात...",phy
2,"तर , थ्रेडच्या परतीच्या स्थितीस पास करण्यासाठी...",cse
3,आपण लोक शोधत आहात जे आपल्यासाठी काहीतरी करू शक...,com_tech
4,लिनक्स कर्नल अनुसूचीतकरणामध्ये अशी तंत्र असण्य...,cse
...,...,...
3775,"नंतर वृद्धत्व , व्हॉट मोठ्या प्रमाणात फ्रॉन 12...",bioche
3776,आणि मग सुद्धा आपल्याला काही सेकंदांनंतर माहित ...,phy
3777,"तर , आपण ELF शीर्षलेखासह प्रारंभ करू .",cse
3778,"तर , त्या क्रॉस्टची जाडी आहे .",phy


In [4]:
x_train = training_data.text.apply(lambda x: preprocess_data(x)).values.tolist()
y_train = training_data.label.values.tolist()
x_val = val_data.text.apply(lambda x: preprocess_data(x)).values.tolist()
y_val = val_data.label.values.tolist()
print(len(x_train))
print(len(y_train))
print(len(x_val))
print(len(y_val))

41997
41997
3780
3780


In [5]:
y_train, y_val = label_encoder(y_train, y_val)

In [6]:
y_train = y_train.reshape(y_train.shape[0], -1)
y_val = y_val.reshape(y_val.shape[0], -1)
print(y_train.shape)
print(y_val.shape)

(41997, 1)
(3780, 1)


In [7]:
pad_len=50
padding_type='post'
truncating_type='post'

In [8]:
tokenizer, x_train_padded, x_val_padded = tokenizer_and_pad_training(x_train, 
                                                                     x_val, 
                                                                     pad_len, 
                                                                     padding_type, 
                                                                     truncating_type)

In [ ]:
dump(tokenizer, "../tokenizers/DL-tokenizer-100.pk1")

In [9]:
print(x_train_padded.shape)
print(x_val_padded.shape)

(41997, 50)
(3780, 50)


In [10]:
vocab = tokenizer.word_index

In [11]:
start = process_time()
embedding_path1 = "/home/eastwind/word-embeddings/fasttext/indicnlp.ft.mr.300.vec"
embedding_matrix1 = get_embedding_matrix(embedding_path1, vocab)
end = process_time()
print("Total time taken: ", end-start)
embedding_matrix1.shape

29198
Total time taken:  7.752140834


(51791, 300)

In [12]:
input_dim = embedding_matrix1.shape[0]
embedding_dim = 300
input_len = pad_len
print("Input vocab: ", input_dim)
print("Embedding dimensions: ", embedding_dim)
print("Input sentence dimensions: ", input_len)

Input vocab:  51791
Embedding dimensions:  300
Input sentence dimensions:  50


In [13]:
myCB = myCallbacks(metrics='acc', threshold=0.95)

# Feed-Forward Neural Network Model

In [14]:
myDNN = create_model_DNN(input_dim, 
                         embedding_dim, 
                         embedding_matrix1, 
                         input_len, 
                         trainable=False, 
                         n1=64, n2=32, 
                         kr=l2(0.015))
myDNN.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          15537300  
_________________________________________________________________
global_max_pooling1d (Global (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                19264     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 132       
Total params: 15,558,776
Trainable params: 21,476
Non-trainable params: 15,537,300
_________________________________________________________________


In [14]:
myDNN = Sequential()
embedding_layer = myDNN.add(
    Embedding(input_dim=input_dim, output_dim=embedding_dim, weights=[embedding_matrix1], input_length=pad_len,
              trainable=trainable))
myDNN.add(GlobalMaxPooling1D())
myDNN.add(Dense(n1, kernel_regularizer=kr, bias_regularizer=br, activation='relu'))
myDNN.add(Dense(n2, kernel_regularizer=kr, bias_regularizer=br, activation='relu'))
myDNN.add(Dense(4, activation='softmax'))
myDNN.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['acc'])
myDNN.summary()

NameError: name 'embedding_matrix' is not defined

In [15]:
history1 = myDNN.fit(x_train_padded, 
                     y_train, 
                     epochs=40, 
                     batch_size=32, 
                     verbose=1, 
                     validation_data = (x_val_padded, y_val), 
                     callbacks=[myCB])

Epoch 1/40
1313/1313 [==============================] - 19s 14ms/step - loss: 1.2284 - acc: 0.6293 - val_loss: 0.9797 - val_acc: 0.6810
Epoch 2/40
1313/1313 [==============================] - 19s 15ms/step - loss: 0.9559 - acc: 0.6928 - val_loss: 0.9591 - val_acc: 0.6812
Epoch 3/40
1313/1313 [==============================] - 20s 16ms/step - loss: 0.9202 - acc: 0.7010 - val_loss: 0.9472 - val_acc: 0.6704
Epoch 4/40
1313/1313 [==============================] - 11s 8ms/step - loss: 0.8968 - acc: 0.7046 - val_loss: 0.8958 - val_acc: 0.7135
Epoch 5/40
1313/1313 [==============================] - 12s 9ms/step - loss: 0.8836 - acc: 0.7067 - val_loss: 0.8630 - val_acc: 0.7132
Epoch 6/40
1313/1313 [==============================] - 12s 9ms/step - loss: 0.8669 - acc: 0.7096 - val_loss: 0.8429 - val_acc: 0.7257
Epoch 7/40
1313/1313 [==============================] - 12s 9ms/step - loss: 0.8563 - acc: 0.7119 - val_loss: 0.8729 - val_acc: 0.7029
Epoch 8/40
1313/1313 [==============================

KeyboardInterrupt: 

In [ ]:
plot_curves(history1)

In [ ]:
name = "FFNN/FFNN-n1.64-n2.32-kr0015-vacc8585"
myDNN.save("../models/"+name+".h5")
# serialize model to JSON
model_json = myDNN.to_json()
with open("../models/"+name+".json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
myDNN.save_weights("../models/"+name+"_weights.h5")

In [ ]:
myDNN = model_load("../models/FFNN/FFNN-n1.64-n2.32-kr0015-vacc8585.h5")

In [ ]:
results1 = np.argmax(myDNN.predict(x_val_padded), axis=-1)

In [ ]:
acc, precision, recall, f1 = classification_report(y_val, results1)
print("Validation Accuracy: ", acc)
print("\nPrecision: ", precision)
print("Average Precision: ", np.mean(precision))
print("\nRecall: ", recall)
print("Average nRecall: ", np.mean(recall))
print("\nF1-Score: ", f1)
print("Average F1-Score: ", np.mean(f1))

# CNN Model

In [16]:
myCNN = create_model_CNN(input_dim, 
                         embedding_dim, 
                         embedding_matrix1, input_len, 
                         trainable=False, 
                         n1=64, k=5, 
                         d=0.25, kr=l2(0.02))
myCNN.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          15537300  
_________________________________________________________________
conv1d (Conv1D)              (None, 96, 64)            96064     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 15,633,624
Trainable params: 96,324
Non-trainable params: 15,537,300
_________________________________________________________________


In [ ]:
history2 = myCNN.fit(x_train_padded, 
                     y_train, 
                     epochs=40, 
                     batch_size=64, 
                     verbose=1, 
                     validation_data = (x_val_padded, y_val), 
                     callbacks=[myCB])

Epoch 1/40
657/657 [==============================] - 47s 71ms/step - loss: 1.0700 - acc: 0.7174 - val_loss: 0.8205 - val_acc: 0.7661
Epoch 2/40
657/657 [==============================] - 55s 83ms/step - loss: 0.8570 - acc: 0.7537 - val_loss: 0.8124 - val_acc: 0.7653
Epoch 3/40
657/657 [==============================] - 50s 76ms/step - loss: 0.8484 - acc: 0.7599 - val_loss: 0.8264 - val_acc: 0.7577
Epoch 4/40
657/657 [==============================] - 54s 82ms/step - loss: 0.8421 - acc: 0.7628 - val_loss: 0.8077 - val_acc: 0.7799
Epoch 5/40
657/657 [==============================] - 50s 77ms/step - loss: 0.8375 - acc: 0.7665 - val_loss: 0.8010 - val_acc: 0.7720
Epoch 6/40
657/657 [==============================] - 52s 79ms/step - loss: 0.8379 - acc: 0.7648 - val_loss: 0.7986 - val_acc: 0.7844
Epoch 7/40
657/657 [==============================] - 60s 92ms/step - loss: 0.8310 - acc: 0.7687 - val_loss: 0.8056 - val_acc: 0.7775
Epoch 8/40
657/657 [==============================] - 54s 82ms

In [ ]:
plot_curves(history2)

In [ ]:
name ="CNN/CNN-n1.64n2.64-kr0015-vacc8632"
myCNN.save("../models/"+name+".h5")
# serialize model to JSON
model_json = myCNN.to_json()
with open("../models/"+name+".json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
myCNN.save_weights("../models/"+name+"_weights.h5")

In [ ]:
myCNN = model_load("../models/CNN/CNN-n1.64n2.64-kr0015-vacc8632.h5")

In [ ]:
results2 = np.argmax(myCNN.predict(x_val_padded), axis=-1)

In [ ]:
acc, precision, recall, f1 = classification_report(y_val, results2)
print("Validation Accuracy: ", acc)
print("\nPrecision: ", precision)
print("Average Precision: ", np.mean(precision))
print("\nRecall: ", recall)
print("Average nRecall: ", np.mean(recall))
print("\nF1-Score: ", f1)
print("Average F1-Score: ", np.mean(f1))

# Bi-LSTM Model

In [ ]:
myLSTM = create_model_LSTM(input_dim, 
                           embedding_dim, 
                           embedding_matrix1, 
                           input_len, 
                           trainable=True, 
                           n1=32, n2=32, 
                           d=0.3)
myLSTM.summary()

In [ ]:
history3 = myLSTM.fit(x_train_padded, 
                      y_train, 
                      epochs=40, 
                      batch_size=64, 
                      verbose=1, 
                      validation_data = (x_val_padded, y_val), 
                      callbacks=[myCB])

In [ ]:
plot_curves(history3)

In [ ]:
name= "LSTM/Bi-LST-n1.64-n2.64-kr0015-vacc868"
myLSTM.save("../models/"+name+".h5")
# serialize model to JSON
model_json = myLSTM.to_json()
with open("../models/"+name+".json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
myLSTM.save_weights("../models/"+name+"_weights.h5")

In [ ]:
myLSTM = model_load("../models/LSTM/Bi-LST-n1.64-n2.64-kr0015-vacc868.h5")

In [ ]:
results3 = np.argmax(myLSTM.predict(x_val_padded), axis=-1)

In [ ]:
acc, precision, recall, f1 = classification_report(y_val, results3)
print("Validation Accuracy: ", acc)
print("\nPrecision: ", precision)
print("Average Precision: ", np.mean(precision))
print("\nRecall: ", recall)
print("Average nRecall: ", np.mean(recall))
print("\nF1-Score: ", f1)
print("Average F1-Score: ", np.mean(f1))